# Training

# Import packages

In [31]:
import pandas as pd
from pathlib import Path
import numpy as np
import ast

from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint

from chemprop import data, featurizers, models, nn
from chemprop.utils import make_mol

# Change data inputs here

In [32]:
chemprop_dir = Path.cwd().parent
input_path = chemprop_dir / "tests" / "data" / "mixed_regression_input.csv" # path to your data .csv file
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles' # name of the column containing SMILES strings
target_columns = ['molecule', 'atom', 'bond'] # list of names of the columns containing targets

In [33]:
df_input = pd.read_csv(input_path)
df_input

,smiles,molecule,atom,bond
0,CC,1.0,"[1,2]",[1]
1,CCC,2.0,"[1,2,3]","[1,2]"
2,CCCO,3.0,"[1,2,3,5]","[1,2,3]"
3,CCOO,4.0,"[1,2,3,4]","[1,2,3]"
4,COO,5.0,"[1,3,5]","[1,2]"
5,CCOOO,6.0,"[1,7,3,4,5]","[1,3,2,4]"
6,COOO,7.0,"[2,5,3,2]","[2,3,4]"
7,CO,8.0,"[1,3]",[2]
8,CCO,9.0,"[1,3,5]","[2,3]"
9,OO,10.0,"[5,7]",[3]


## Get SMILES and targets

In [34]:
smis = df_input.loc[:, smiles_column].values
ys = df_input.loc[:, target_columns]

In [35]:
smis[:2] # show first 2 SMILES strings

array(['CC', 'CCC'], dtype=object)

In [36]:
ys[:5] # show first 5 molecule targets

,molecule,atom,bond
0,1.0,"[1,2]",[1]
1,2.0,"[1,2,3]","[1,2]"
2,3.0,"[1,2,3,5]","[1,2,3]"
3,4.0,"[1,2,3,4]","[1,2,3]"
4,5.0,"[1,3,5]","[1,2]"


In [37]:
import numpy as np

flag, mol_Y, atom_Y, bond_Y = [], [], [], []
for column in target_columns:
    index = 0
    column_type = df_input.iloc[index][column]
    if isinstance(column_type, float):
        for molecule in range(len(df_input)):
            mol_Y.append([df_input.iloc[molecule][column]])
        flag.append("mol")
    else:
        column_mol = make_mol(df_input.iloc[index][smiles_column], False, False)
        column_type = ast.literal_eval(column_type)
        while index < len(df_input) and column_mol.GetNumAtoms() == column_mol.GetNumBonds():
            index += 1
            column_mol = make_mol(df_input.iloc[index][smiles_column], False, False)
        column_type = ast.literal_eval(df_input.iloc[index][column])
        flag.append("atom") if len(column_type) == column_mol.GetNumAtoms() else flag.append("bond")
        
for molecule in range(len(df_input)):
    atom_list_props = []
    bond_list_props = []
    for prop in range(len(target_columns)):
        if flag[prop] == "mol":
            continue
        np_prop = np.array(ast.literal_eval(df_input.iloc[molecule][target_columns[prop]]))
        np_prop = np.expand_dims(np_prop, axis=1)
        atom_list_props.append(np_prop) if flag[prop] == "atom" else bond_list_props.append(np_prop)
    if len(atom_list_props) > 0:
        atom_Y.append(np.hstack(atom_list_props))
    if len(bond_list_props) > 0:
        bond_Y.append(np.hstack(bond_list_props))

print(mol_Y)

[[1.0], [2.0], [3.0], [4.0], [5.0], [6.0], [7.0], [8.0], [9.0], [10.0]]


## Get molecule datapoints

In [38]:
all_data = []

mol_data = []
for smi, y in zip(smis, mol_Y):
    mol_data.append(data.MoleculeDatapoint.from_smi(smi, y, keep_h=True))

atom_data = []
for smi, y in zip(smis, atom_Y):
    atom_data.append(data.MoleculeDatapoint.from_smi(smi, y, keep_h=True))

bond_data = []
for smi, y in zip(smis, bond_Y):
    bond_data.append(data.MoleculeDatapoint.from_smi(smi, y, keep_h=True))

all_data.append(mol_data)
all_data.append(atom_data)
all_data.append(bond_data)

## Perform data splitting for training, validation, and testing

In [39]:
# available split types
list(data.SplitType.keys())

['SCAFFOLD_BALANCED',
 'RANDOM_WITH_REPEATED_SMILES',
 'RANDOM',
 'KENNARD_STONE',
 'KMEANS']

In [40]:
mol = [d.mol for d in all_data[0]]

train_indices, val_indices, test_indices = data.make_split_indices(mol, "random", (0.6, 0.2, 0.2))
print(all_data)
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)


[[MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x28498af10>, y=[1.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='CC', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x1771044a0>, y=[2.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='CCC', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x177104c80>, y=[3.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='CCCO', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x1771040b0>, y=[4.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='CCOO', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x1771075a0>, y=[5.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='COO', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=

## Get AtomDataset

In [41]:
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

train_dsets = []
train_dsets.append(data.MoleculeDataset(train_data[0][0], featurizer))
train_dsets.append(data.AtomDataset(train_data[0][1], featurizer))
train_dsets.append(data.BondDataset(train_data[0][2], featurizer))
print(train_dsets[0])
mol_scaler = train_dsets[0].normalize_targets()
atom_scaler = train_dsets[1].normalize_targets()
bond_scaler = train_dsets[2].normalize_targets()
train_dset = data.MolAtomBondDataset(train_dsets[0], train_dsets[1], train_dsets[2])

MoleculeDataset(data=[MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x177107920>, y=[9.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='CCO', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x1771075a0>, y=[5.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='COO', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x177107a70>, y=[10.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='OO', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x1771044a0>, y=[2.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='CCC', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x177107df0>, y=[7.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='COOO', V_f=None, E_f=None, V_d=None, E_d=None), Mo

In [42]:
val_dsets = []
val_dsets.append(data.MoleculeDataset(val_data[0][0], featurizer))
val_dsets.append(data.AtomDataset(val_data[0][1], featurizer))
val_dsets.append(data.BondDataset(val_data[0][2], featurizer))
val_dsets[0].normalize_targets(mol_scaler)
val_dsets[1].normalize_targets(atom_scaler)
val_dsets[2].normalize_targets(bond_scaler)
val_dset = data.MolAtomBondDataset(val_dsets[0], val_dsets[1], val_dsets[2])

test_dsets = []
test_dsets.append(data.MoleculeDataset(test_data[0][0], featurizer))
test_dsets.append(data.AtomDataset(test_data[0][1], featurizer))
test_dsets.append(data.BondDataset(test_data[0][2], featurizer))
test_dset = data.MolAtomBondDataset(test_dsets[0], test_dsets[1], test_dsets[2])

all_dsets = []
all_dsets.append(data.MoleculeDataset(all_data[0], featurizer))
all_atom_data = data.AtomDataset(all_data[1], featurizer)
all_dsets.append(all_atom_data)
atom_slices = all_atom_data._slices
all_bond_data = data.BondDataset(all_data[2], featurizer)
all_dsets.append(all_bond_data)
bond_slices = all_bond_data._slices
all_dset = data.MolAtomBondDataset(all_dsets[0], all_dsets[1], all_dsets[2])

## Get DataLoader

In [43]:
train_loader = data.build_dataloader(train_dset, num_workers=num_workers, shuffle=False)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)
all_loader = data.build_dataloader(all_dset, num_workers=num_workers, shuffle=False)

# Change Message-Passing Neural Network (MPNN) inputs here

## Message Passing
A `Message passing` constructs molecular graphs using message passing to learn node-level hidden representations.

Options are `mp = nn.BondMessagePassing()` or `mp = nn.AtomMessagePassing()`

In [44]:
mp = nn.MixedBondMessagePassing() #include why aggregation isn't used
mp

MixedBondMessagePassing(
  (W_i): Linear(in_features=86, out_features=300, bias=False)
  (W_h): Linear(in_features=300, out_features=300, bias=False)
  (W_o): Linear(in_features=372, out_features=300, bias=True)
  (W_o_b): Linear(in_features=314, out_features=300, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (tau): ReLU()
  (V_d_transform): Identity()
  (E_d_transform): Identity()
  (graph_transform): Identity()
)

## Feed-Forward Network (FFN)

A `FFN` takes the aggregated representations and make target predictions.

Available options can be found in `nn.PredictorRegistry`.

For regression:
- `ffn = nn.RegressionFFN()`
- `ffn = nn.MveFFN()`
- `ffn = nn.EvidentialFFN()`

For classification:
- `ffn = nn.BinaryClassificationFFN()`
- `ffn = nn.BinaryDirichletFFN()`
- `ffn = nn.MulticlassClassificationFFN()`
- `ffn = nn.MulticlassDirichletFFN()`

For spectral:
- `ffn = nn.SpectralFFN()` # will be available in future version

In [45]:
print(nn.PredictorRegistry)

ClassRegistry {
    'regression': <class 'chemprop.nn.predictors.RegressionFFN'>,
    'regression-mve': <class 'chemprop.nn.predictors.MveFFN'>,
    'regression-evidential': <class 'chemprop.nn.predictors.EvidentialFFN'>,
    'regression-quantile': <class 'chemprop.nn.predictors.QuantileFFN'>,
    'classification': <class 'chemprop.nn.predictors.BinaryClassificationFFN'>,
    'classification-dirichlet': <class 'chemprop.nn.predictors.BinaryDirichletFFN'>,
    'multiclass': <class 'chemprop.nn.predictors.MulticlassClassificationFFN'>,
    'multiclass-dirichlet': <class 'chemprop.nn.predictors.MulticlassDirichletFFN'>,
    'spectral': <class 'chemprop.nn.predictors.SpectralFFN'>
}


In [46]:
mol_output_transform = nn.UnscaleTransform.from_standard_scaler(mol_scaler)
atom_output_transform = nn.UnscaleTransform.from_standard_scaler(atom_scaler)
bond_output_transform = nn.UnscaleTransform.from_standard_scaler(bond_scaler)

In [47]:
mol_ffn = nn.RegressionFFN(output_transform=mol_output_transform, n_tasks=1)
atom_ffn = nn.RegressionFFN(output_transform=atom_output_transform, n_tasks=1)
bond_ffn = nn.RegressionFFN(output_transform=bond_output_transform, input_dim=600, n_tasks=1)

## Batch Norm
A `Batch Norm` normalizes the outputs of the aggregation by re-centering and re-scaling.

Whether to use batch norm

In [48]:
batch_norm = True

## Metrics
`Metrics` are the ways to evaluate the performance of model predictions.

Available options can be found in `metrics.MetricRegistry`, including

In [49]:
print(nn.metrics.MetricRegistry)

ClassRegistry {
    'mse': <class 'chemprop.nn.metrics.MSE'>,
    'mae': <class 'chemprop.nn.metrics.MAE'>,
    'rmse': <class 'chemprop.nn.metrics.RMSE'>,
    'bounded-mse': <class 'chemprop.nn.metrics.BoundedMSE'>,
    'bounded-mae': <class 'chemprop.nn.metrics.BoundedMAE'>,
    'bounded-rmse': <class 'chemprop.nn.metrics.BoundedRMSE'>,
    'r2': <class 'chemprop.nn.metrics.R2Score'>,
    'binary-mcc': <class 'chemprop.nn.metrics.BinaryMCCMetric'>,
    'multiclass-mcc': <class 'chemprop.nn.metrics.MulticlassMCCMetric'>,
    'roc': <class 'chemprop.nn.metrics.BinaryAUROC'>,
    'prc': <class 'chemprop.nn.metrics.BinaryAUPRC'>,
    'accuracy': <class 'chemprop.nn.metrics.BinaryAccuracy'>,
    'f1': <class 'chemprop.nn.metrics.BinaryF1Score'>
}


In [50]:
metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()] # Only the first metric is used for training and early stopping

## Constructs MPNN

In [51]:
agg = nn.MeanAggregation()
mol_atom_bond_mpnn = models.MolAtomBondMPNN(mp, agg, mol_ffn, atom_ffn, bond_ffn, batch_norm, metric_list)

mol_atom_bond_mpnn

/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'mol_predictor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['mol_predictor'])`.
/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'atom_predictor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['atom_predictor'])`.
/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'bond_predictor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['bond_predictor'])`.


MolAtomBondMPNN(
  (message_passing): MixedBondMessagePassing(
    (W_i): Linear(in_features=86, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=372, out_features=300, bias=True)
    (W_o_b): Linear(in_features=314, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (E_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictors): ModuleList(
    (0-1): 2 x RegressionFFN(
      (ffn): MLP(
        (0): Sequential(
          (0): Linear(in_features=300, out_features=300, bias=True)
        )
        (1): Sequential(
          (0): ReLU()
          (1): Dropout(p=0.0, inplace=False)
          (2): Linear(in_features=300, out_features=1, bias=True)
        )
      )
      (criterion): MSE(task_weights=[[1.0]

# Set up trainer

In [52]:
# Configure model checkpointing
checkpointing = ModelCheckpoint(
    "checkpoints",  # Directory where model checkpoints will be saved
    "best-{epoch}-{val_loss:.2f}",  # Filename format for checkpoints, including epoch and validation loss
    "val_loss",  # Metric used to select the best checkpoint (based on validation loss)
    mode="min",  # Save the checkpoint with the lowest validation loss (minimization objective)
    save_last=True,  # Always save the most recent checkpoint, even if it's not the best
)

trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="cpu",
    devices=1,
    max_epochs=20, # number of epochs to train for
    callbacks=[checkpointing],
)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


# Start training

In [53]:
trainer.fit(mol_atom_bond_mpnn, train_loader, val_loader)

/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/brianli/Documents/chemprop/examples/checkpoints exists and is not empty.
Loading `train_dataloader` to estimate number of stepping batches.
/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.

  | Name            | Type                    | Params | Mode 
--------------------------------------------------------------------
0 | message_passing | MixedBondMessagePassing | 322 K  | train
1 | agg             | MeanAggregation         | 0      | train
2 | bn              | BatchNorm1d             | 600    | train
3 | predictors      | ModuleList              | 361 K  | train
4 | X_d_transform

/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 0: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 68.03it/s]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|████████████████████| 1/1 [00:00<00:00, 76.66it/s, val_loss=4.820]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|████████████████████| 1/1 [00:00<00:00, 74.71it/s, val_loss=4.700]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|████████████████████| 1/1 [00:00<00:00, 64.52it/s, val_loss=4.480]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|████████████████████| 1/1 [00:00<00:00, 83.29it/s, val_loss=4.290]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|████████████████████| 1/1 [00:00<00:00, 87.98it/s, val_loss=4.140]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 6: 100%|██████████████

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|███████████████████| 1/1 [00:00<00:00, 23.02it/s, val_loss=1.660]


# Test results

In [54]:
results = trainer.test(mol_atom_bond_mpnn, test_loader)

/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████████████████| 1/1 [00:00<00:00, 181.22it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     atom_mol_test/mae     │    1.1712143421173096     │
│    atom_mol_test/rmse     │    1.6336032152175903     │
│  bond_atom_mol_test/mae   │    1.1712143421173096     │
│  bond_atom_mol_test/rmse  │    1.6336032152175903     │
│       mol_test/mae        │    1.1712143421173096     │
│       mol_test/rmse       │    1.6336032152175903     │
└───────────────────────────┴───────────────────────────┘

# Predictions

In [61]:
from chemprop.models import load_mixed_model
import torch

mol_individual_preds = []
atom_individual_preds = []
bond_individual_preds = []
model = load_mixed_model(checkpointing.best_model_path)
print(model.state_dict().keys())
trainer = pl.Trainer(
    logger=False,
    enable_progress_bar=True,
    accelerator="cpu",
    devices=1,
)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


odict_keys(['message_passing.W_i.weight', 'message_passing.W_h.weight', 'message_passing.W_o.weight', 'message_passing.W_o.bias', 'message_passing.W_o_b.weight', 'message_passing.W_o_b.bias', 'bn.weight', 'bn.bias', 'bn.running_mean', 'bn.running_var', 'bn.num_batches_tracked', 'predictors.0.ffn.0.0.weight', 'predictors.0.ffn.0.0.bias', 'predictors.0.ffn.1.2.weight', 'predictors.0.ffn.1.2.bias', 'predictors.0.criterion.task_weights', 'predictors.0.output_transform.mean', 'predictors.0.output_transform.scale', 'predictors.1.ffn.0.0.weight', 'predictors.1.ffn.0.0.bias', 'predictors.1.ffn.1.2.weight', 'predictors.1.ffn.1.2.bias', 'predictors.1.criterion.task_weights', 'predictors.1.output_transform.mean', 'predictors.1.output_transform.scale', 'predictors.2.ffn.0.0.weight', 'predictors.2.ffn.0.0.bias', 'predictors.2.ffn.1.2.weight', 'predictors.2.ffn.1.2.bias', 'predictors.2.criterion.task_weights', 'predictors.2.output_transform.mean', 'predictors.2.output_transform.scale', 'metrics.0.0.

In [56]:
predss = trainer.predict(model, all_loader)
print(predss[0])
mol_individual_preds.append(torch.concat([predss[0][0]], 0))
atom_individual_preds.append(torch.concat([predss[0][1]], 0))
bond_individual_preds.append(torch.concat([predss[0][2]], 0))

mol_average_preds = torch.mean(torch.stack(mol_individual_preds).float(), dim=0)
atom_average_preds = torch.mean(torch.stack(atom_individual_preds).float(), dim=0)
bond_average_preds = torch.mean(torch.stack(bond_individual_preds).float(), dim=0)

test_path = chemprop_dir / "tests" / "data" / "mixed_regression_input.csv"
df_test = pd.read_csv(test_path, header="infer", index_col=False)

/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|███████████████████| 1/1 [00:00<00:00, 288.68it/s]
[tensor([[4.1460],
        [3.9199],
        [6.9743],
        [6.6796],
        [6.1004],
        [6.8846],
        [6.7616],
        [7.5233],
        [7.6099],
        [8.9597]]), tensor([[2.5122],
        [2.5122],
        [2.4572],
        [2.4596],
        [2.4572],
        [2.4783],
        [2.4314],
        [2.7524],
        [3.4947],
        [2.4245],
        [2.6064],
        [3.4937],
        [3.5837],
        [2.5472],
        [3.4379],
        [3.6026],
        [2.4275],
        [2.6359],
        [3.3587],
        [3.5838],
        [3.5249],
        [2.5740],
        [3.3328],
        [3.5938],
        [3.5250],
        [2.6940],
        [3.4407],
        [2.4128],
        [2.7418],
        [3.5024],
        [3.6684],
        [3.6684]]), tensor([[1.4151],
        [1.4151],
        [1.5157],
        [1.5184],
        [1.5184],
        [1.5157],
        [1.5397],
        [1.5448],
        [1.635

## Loaded Model

In [57]:
model

MolAtomBondMPNN(
  (message_passing): MixedBondMessagePassing(
    (W_i): Linear(in_features=86, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=372, out_features=300, bias=True)
    (W_o_b): Linear(in_features=314, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (E_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictors): ModuleList(
    (0-1): 2 x RegressionFFN(
      (ffn): MLP(
        (0): Sequential(
          (0): Linear(in_features=300, out_features=300, bias=True)
        )
        (1): Sequential(
          (0): ReLU()
          (1): Dropout(p=0.0, inplace=False)
          (2): Linear(in_features=300, out_features=1, bias=True)
        )
      )
      (criterion): MSE(task_weights=[[1.0]

In [58]:
target_cols = df_test.columns.tolist()
mol_cols, atom_cols, bond_cols = [], [], []
for i in range(1, len(target_cols)):
    index = 0
    column_type = df_test.iloc[index][target_cols[i]]
    if isinstance(column_type, float):
        mol_cols.append(i)
    else:
        column_mol = make_mol(df_test.iloc[index][target_cols[0]], False, False)
        column_type = ast.literal_eval(column_type)
        while index < len(df_test) and column_mol.GetNumAtoms() == column_mol.GetNumBonds():
            index += 1
            column_mol = make_mol(df_test.iloc[index][target_cols[0]])
        column_type = ast.literal_eval(df_test.iloc[index][target_cols[i]])
        atom_cols.append(i) if len(column_type) == column_mol.GetNumAtoms() else bond_cols.append(i)

In [59]:
df_test.iloc[:, mol_cols] = mol_average_preds.tolist()

for i in range(len(df_test)):
    first_atom = atom_slices.index(i)
    last_atom = first_atom + atom_slices.count(i)
    atom_preds = atom_average_preds[first_atom:last_atom]
    df_test.iloc[i, atom_cols] = [str(atom_preds[:,j].tolist()) for j in range(len(atom_cols))]

    first_bond = bond_slices.index(i)
    last_bond = first_bond + bond_slices.count(i)
    bond_preds = bond_average_preds[first_atom:last_atom]
    df_test.iloc[i, bond_cols] = [str(bond_preds[:,j].tolist()) for j in range(len(bond_cols))]

#torch.split(average_preds, split_size_or_sections=torch.bincount(torch.tensor(slices)).tolist(), dim=0)

output_path = chemprop_dir / "tests" / "data" / "mixed_regression_output.csv"
if output_path.suffix == ".pkl":
    df_test = df_test.reset_index(drop=True)
    df_test.to_pickle(output_path)
else:
    df_test.to_csv(output_path, index=False)

df_test

# do the reordering of target columns outside here in jupyter notebook but inside parsing.py in the cli

,smiles,molecule,atom,bond
0,CC,4.145959,"[2.512220859527588, 2.512220859527588]","[1.4150546789169312, 1.4150545597076416]"
1,CCC,3.919910,"[2.4572410583496094, 2.4596452713012695, 2.457...","[1.5156538486480713, 1.5184483528137207, 1.518..."
2,CCCO,6.974278,"[2.478332281112671, 2.4314184188842773, 2.7524...","[1.5156538486480713, 1.5397183895111084, 1.544..."
3,CCOO,6.679646,"[2.42449951171875, 2.6063575744628906, 3.49367...","[1.6312777996063232, 2.41660213470459, 2.42530..."
4,COO,6.100406,"[2.547196388244629, 3.437858819961548, 3.60258...","[1.529393196105957, 2.206265926361084, 2.20596..."
5,CCOOO,6.884593,"[2.427492380142212, 2.635878562927246, 3.35873...","[2.933560609817505, 2.940661668777466, 1.96759..."
6,COOO,6.761596,"[2.57401704788208, 3.3328075408935547, 3.59375...","[2.90136456489563, 1.5132120847702026, 1.52044..."
7,CO,7.523325,"[2.6939995288848877, 3.4407336711883545]","[2.3256585597991943, 2.821397304534912]"
8,CCO,7.609945,"[2.4128382205963135, 2.741804838180542, 3.5023...","[2.8188865184783936, 2.972829818725586, 2.9858..."
9,OO,8.959720,"[3.6683731079101562, 3.6683731079101562]","[2.0887036323547363, 2.087083339691162]"
